In [8]:
# 처음 한번만 받으면 됨
# !wget http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
# !wget http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r101_fpn_2x_coco/faster_rcnn_r101_fpn_2x_coco_bbox_mAP-0.398_20200504_210455-1d2dac9c.pth
# !wget http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_2x_coco/faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth

In [9]:
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

In [3]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_fpn_2x_coco.py')

PREFIX = '../../input/data/'


# dataset 바꾸기
cfg.data.train.classes = classes
# cfg.data.train.img_prefix = PREFIX
# cfg.data.train.ann_file = PREFIX + 'train.json'
cfg.data.train.pipeline[2]['img_scale'] = (1333, 800)

cfg.data.train.img_prefix = [PREFIX, PREFIX]
cfg.data.train.ann_file = [PREFIX + 'train.json', PREFIX + 'coco_pseudo_train_vfnet_6.json']


cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
cfg.data.val.pipeline[1]['img_scale'] = (1333, 800)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (1333, 800)

cfg.data.samples_per_gpu = 8
cfg.data.workers_per_gpu = 4

cfg.seed=2020
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/sudo_faster_rcnn_r50_fpn_1x_trash_warmup_fix'

cfg.load_from = 'faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth'

cfg.checkpoint_config = dict(max_keep_ckpts=2, interval=1)


cfg.model.roi_head.bbox_head.num_classes = 11

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)

cfg.log_config = dict(  # config to register logger hook
    interval=40,  # Interval to print the log
    hooks=[
        dict(type='TensorboardLoggerHook'),  # The Tensorboard logger is also supported
        dict(type='TextLoggerHook')
    ])


##################### 추가 사항 ##########################
# cfg.work_dir = './work_dirs/faster_rcnn_2x_batch8_wontrans_pseudo2_s30c50'
# cfg.load_from = 'epoch_12-Copy1.pth'
# cfg.runner = dict(type='EpochBasedRunner', max_epochs=12)
# cfg.lr_config = dict(
#     policy='step',
#     warmup='linear',
#     warmup_iters= int(1300/cfg.data.samples_per_gpu),
#     warmup_ratio=0.001,
#     step=[8, 11, 31])
# lr_config = dict(
#     policy='step',
#     warmup='linear',
#     warmup_iters=500,
#     warmup_ratio=0.001,
#     step=[8, 11])

# 1.5배 늘림 (2X에서) 왜냐하면 augmentation 많이 들어가서 학습이 더 걸린다고 생각했고, 과적합을 막아준다고 판단
# cfg.lr_config = dict(
#     policy='step',
#     warmup='linear',
#     warmup_iters=500,
#     warmup_ratio=0.001,
#     step=[24, 33])
# cfg.runner = dict(type='EpochBasedRunner', max_epochs=36)

# cfg.albu_train_transforms = [
#     dict(type='HorizontalFlip',p=.25),
#     dict(type='ShiftScaleRotate', p=.25),
#     dict(
#         type='RandomResizedCrop',
#         height=512, 
#         width=512, 
#         scale=[0.75, 1], 
#         p=0.5
#         ),
#     dict(
#         type='RandomBrightnessContrast',
#         brightness_limit=[0.1, 0.3],
#         contrast_limit=[0.1, 0.3],
#         p=0.2),
#     dict(
#         type='OneOf',
#         transforms=[
#             dict(
#                 type='RGBShift',
#                 r_shift_limit=10,
#                 g_shift_limit=10,
#                 b_shift_limit=10,
#                 p=1.0),
#             dict(
#                 type='HueSaturationValue',
#                 hue_shift_limit=20,
#                 sat_shift_limit=30,
#                 val_shift_limit=20,
#                 p=1.0)
#         ],
#         p=0.1),
#     dict(
#         type='OneOf',
#         transforms=[
#             dict(type='Blur', blur_limit=3, p=1.0),
#             dict(type='MedianBlur', blur_limit=3, p=1.0)
#         ],
#         p=0.1),
# ]
    
# # Mosaic
# cfg.train_pipeline = [
#     dict(type='LoadImageFromFile'),
#     dict(type='LoadAnnotations', with_bbox=True),
#     dict(type='Mosaic'),
#     dict(
#         type='Albu',
#         transforms= cfg.albu_train_transforms,
#         bbox_params=dict(
#             type='BboxParams',
#             format='pascal_voc',
#             label_fields=['gt_labels'],
#             min_visibility=0.0,
#             filter_lost_elements=True),
#         keymap={
#             'img': 'image',
#             'gt_masks': 'masks',
#             'gt_bboxes': 'bboxes'
#         },
#         update_pad_shape=False,
#         skip_img_without_anno=True),
#     dict(type='Resize', img_scale=(1333, 800), keep_ratio=True),
#     dict(type='RandomFlip', flip_ratio=0.5),
#     dict(
#         type='Normalize',
#         mean=[123.675, 116.28, 103.53],
#         std=[58.395, 57.12, 57.375],
#         to_rgb=True),
#     dict(type='Pad', size_divisor=32),
#     dict(type='DefaultFormatBundle'),
#     dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels'])
# ]

# cfg.data.train.pipeline=cfg.train_pipeline

cfg.img_norm_cfg = dict(mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
cfg.albu_train_transforms = [
    dict(
        type="Cutout",
        num_holes=30,
        max_h_size=30,
        max_w_size=30,
        fill_value=cfg.img_norm_cfg["mean"][::-1],
        p=0.1,
    ),
#     albu_example
    dict(
        type='RandomBrightnessContrast',
        brightness_limit=[0.1, 0.3],
        contrast_limit=[0.1, 0.3],
        p=0.2),
    dict(
        type='OneOf',
        transforms=[
            dict(
                type='RGBShift',
                r_shift_limit=10,
                g_shift_limit=10,
                b_shift_limit=10,
                p=1.0),
            dict(
                type='HueSaturationValue',
                hue_shift_limit=20,
                sat_shift_limit=30,
                val_shift_limit=20,
                p=1.0),
            dict(type="RandomGamma"),
            dict(type="CLAHE"),
        ],
        p=0.1),
    dict(type='JpegCompression', quality_lower=85, quality_upper=95, p=0.2),
    dict(type='ChannelShuffle', p=0.1),
    dict(
        type='OneOf',
        transforms=[
            dict(type='Blur', blur_limit=3, p=1.0),
            dict(type='MedianBlur', blur_limit=3, p=1.0),
            dict(type="MotionBlur"),
            dict(type="GaussNoise"),
            dict(type="ImageCompression", quality_lower=75),
        ],
        p=0.1),
    dict(type='RandomRotate90', p=0.2),
]
# cfg.data.train.pipeline.insert(4, dict(
#         type='Albu',
#         transforms=albu_train_transforms,
#         bbox_params=dict(
#             type='BboxParams',
#             format='pascal_voc',
#             label_fields=['gt_labels'],
#             min_visibility=0.0,
#             filter_lost_elements=True),
#         keymap={
#             'img': 'image',
#             'gt_masks': 'masks',
#             'gt_bboxes': 'bboxes'
#         },
#         update_pad_shape=False,
#         skip_img_without_anno=True))

# Mosaic
cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
#     dict(type='Mosaic'),
    dict(type='Resize', img_scale=(1333, 800), keep_ratio=True),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(
        type='Albu',
        transforms= cfg.albu_train_transforms,
        bbox_params=dict(
            type='BboxParams',
            format='pascal_voc',
            label_fields=['gt_labels'],
            min_visibility=0.0,
            filter_lost_elements=True),
        keymap={
            'img': 'image',
            'gt_masks': 'masks',
            'gt_bboxes': 'bboxes'
        },
        update_pad_shape=False,
        skip_img_without_anno=True),
    dict(
        type='Normalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels'])
]

cfg.data.train.pipeline=cfg.train_pipeline

# Transform 1
# cfg.albu_train_transforms = [
#     dict(type='VerticalFlip',p=.25),
#     dict(
#         type='CoarseDropout',
#         max_holes=10,
#         max_height=51, 
#         max_width=51, p=.25
#         ),
#     dict(type='ShiftScaleRotate', p=.25),
#     dict(
#         type='RandomResizedCrop',
#         height=512, 
#         width=512, 
#         scale=[0.75, 1], 
#         p=1
#         ),
# ]

# # Transform 2
# cfg.albu_train_transforms = [
#     dict(
#         type='OneOf',
#         transforms=[
#             dict(type='RandomRotate90',p=.25),
#             dict(type='VerticalFlip',p=.5)
#         ],
#         p=0.1),
#     dict(
#         type='OneOf',
#         transforms=[
#             dict(type='Resize',height=512, width=512, p=.5),
#             dict(type='Resize',height=1024, width=1024, p=.25),
#             dict(type='Resize',height=256, width=256, p=.25)
#         ],
#         p=1),
#     dict(type='Cutout', num_holes=10, max_h_size=51, max_w_size=51, p=.25),
#     dict(type='ShiftScaleRotate', p=0.25),
#     dict(type='RandomResizedCrop', height=512, width=512, scale=[0.75, 1], p=1)
# ]

# cfg.img_norm_cfg = dict(
#     mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)


# cfg.train_pipeline = [
#     dict(type='LoadImageFromFile'),
#     dict(type='LoadAnnotations', with_bbox=True, with_mask=True),
#     dict(
#         type='Albu',
#         transforms= cfg.albu_train_transforms,
#         bbox_params=dict(
#             type='BboxParams',
#             format='pascal_voc',
#             label_fields=['gt_labels'],
#             min_visibility=0.0,
#             filter_lost_elements=True),
#         keymap={
#             'img': 'image',
#             'gt_masks': 'masks',
#             'gt_bboxes': 'bboxes'
#         },
#         update_pad_shape=False,
#         skip_img_without_anno=True),
#     dict(type='Resize', img_scale=(512, 512), keep_ratio=True),
#     dict(type='Pad', size_divisor=32),
#     dict(type='Normalize', **cfg.img_norm_cfg),
#     dict(type='DefaultFormatBundle'),
#     dict(
#         type='Collect',
#         keys=['img', 'gt_bboxes', 'gt_labels', 'gt_masks'],
#         meta_keys=('filename', 'ori_shape', 'img_shape', 'img_norm_cfg',
#                    'pad_shape', 'scale_factor'))
# ]
# cfg.data.train.pipeline = cfg.train_pipeline

# ##################### pirnt #####################################
print(f'Config:\n{cfg.pretty_text}')

Config:
model = dict(
    type='FasterRCNN',
    pretrained='torchvision://resnet50',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch'),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0),
        loss_bbox=dict(typ

In [4]:
model = build_detector(cfg.model)

2021-05-18 13:16:23,122 - mmdet - INFO - load model from: torchvision://resnet50
2021-05-18 13:16:23,123 - mmdet - INFO - Use load_from_torchvision loader
2021-05-18 13:16:23,379 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias



In [5]:
# model

In [6]:
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=3.71s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [ ]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

loading annotations into memory...
Done (t=0.88s)
creating index...


2021-05-18 13:16:31,590 - mmdet - INFO - load checkpoint from faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth
2021-05-18 13:16:31,594 - mmdet - INFO - Use load_from_local loader
2021-05-18 13:16:31,740 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for roi_head.bbox_head.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([12, 1024]).
size mismatch for roi_head.bbox_head.fc_cls.bias: copying a param with shape torch.Size([81]) from checkpoint, the shape in current model is torch.Size([12]).
size mismatch for roi_head.bbox_head.fc_reg.weight: copying a param with shape torch.Size([320, 1024]) from checkpoint, the shape in current model is torch.Size([44, 1024]).
size mismatch for roi_head.bbox_head.fc_reg.bias: copying a param with shape torch.Size([320]) from checkpoint, the shape in current model is torch.Size([44]).
2021-05-18 13:16:31,742 - mmdet - 

index created!


2021-05-18 13:17:16,159 - mmdet - INFO - Epoch [1][40/336]	lr: 1.578e-03, eta: 2:27:53, time: 1.106, data_time: 0.081, memory: 8336, loss_rpn_cls: 0.1434, loss_rpn_bbox: 0.0597, loss_cls: 1.0946, acc: 74.1711, loss_bbox: 0.5085, loss: 1.8063, grad_norm: 6.6547
2021-05-18 13:17:56,868 - mmdet - INFO - Epoch [1][80/336]	lr: 3.177e-03, eta: 2:21:17, time: 1.018, data_time: 0.017, memory: 8336, loss_rpn_cls: 0.0564, loss_rpn_bbox: 0.0431, loss_cls: 0.4396, acc: 88.1567, loss_bbox: 0.4318, loss: 0.9709, grad_norm: 2.2620
2021-05-18 13:18:38,129 - mmdet - INFO - Epoch [1][120/336]	lr: 4.775e-03, eta: 2:19:14, time: 1.032, data_time: 0.017, memory: 8336, loss_rpn_cls: 0.0657, loss_rpn_bbox: 0.0496, loss_cls: 0.4212, acc: 88.8251, loss_bbox: 0.3688, loss: 0.9053, grad_norm: 3.2077
2021-05-18 13:19:18,787 - mmdet - INFO - Epoch [1][160/336]	lr: 6.374e-03, eta: 2:17:23, time: 1.016, data_time: 0.016, memory: 8336, loss_rpn_cls: 0.0665, loss_rpn_bbox: 0.0511, loss_cls: 0.4102, acc: 88.8519, loss_

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 655/655, 14.9 task/s, elapsed: 44s, ETA:     0s

...

2021-05-18 15:11:13,951 - mmdet - INFO - Epoch [18][40/336]	lr: 2.000e-03, eta: 0:38:06, time: 1.100, data_time: 0.082, memory: 8337, loss_rpn_cls: 0.0181, loss_rpn_bbox: 0.0349, loss_cls: 0.1882, acc: 93.7720, loss_bbox: 0.2033, loss: 0.4444, grad_norm: 1.7071
2021-05-18 15:11:55,215 - mmdet - INFO - Epoch [18][80/336]	lr: 2.000e-03, eta: 0:37:27, time: 1.032, data_time: 0.018, memory: 8337, loss_rpn_cls: 0.0216, loss_rpn_bbox: 0.0371, loss_cls: 0.2105, acc: 93.2800, loss_bbox: 0.2208, loss: 0.4899, grad_norm: 1.9742
2021-05-18 15:12:36,391 - mmdet - INFO - Epoch [18][120/336]	lr: 2.000e-03, eta: 0:36:48, time: 1.029, data_time: 0.017, memory: 8337, loss_rpn_cls: 0.0170, loss_rpn_bbox: 0.0311, loss_cls: 0.1748, acc: 94.3195, loss_bbox: 0.1874, loss: 0.4104, grad_norm: 1.7415
2021-05-18 15:13:17,644 - mmdet - INFO - Epoch [18][160/336]	lr: 2.000e-03, eta: 0:36:09, time: 1.031, data_time: 0.017, memory: 8337, loss_rpn_cls: 0.0159, loss_rpn_bbox: 0.0335, loss_cls: 0.1753, acc: 94.2078, l

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 655/655, 14.7 task/s, elapsed: 45s, ETA:     0s

2021-05-18 15:17:04,827 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=0.52s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=4.94s).
Accumulating evaluation results...
DONE (t=0.75s).
Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.254
Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.386
Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.259
Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.060
Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.225
Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.458
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.400
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.400
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.400
Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ]

2021-05-18 15:17:54,967 - mmdet - INFO - Epoch [19][40/336]	lr: 2.000e-03, eta: 0:32:33, time: 1.096, data_time: 0.081, memory: 8337, loss_rpn_cls: 0.0164, loss_rpn_bbox: 0.0331, loss_cls: 0.1755, acc: 94.3439, loss_bbox: 0.1919, loss: 0.4169, grad_norm: 1.7223
2021-05-18 15:18:36,143 - mmdet - INFO - Epoch [19][80/336]	lr: 2.000e-03, eta: 0:31:54, time: 1.029, data_time: 0.017, memory: 8337, loss_rpn_cls: 0.0176, loss_rpn_bbox: 0.0325, loss_cls: 0.1802, acc: 94.1681, loss_bbox: 0.1901, loss: 0.4203, grad_norm: 1.7775
2021-05-18 15:19:17,432 - mmdet - INFO - Epoch [19][120/336]	lr: 2.000e-03, eta: 0:31:15, time: 1.032, data_time: 0.018, memory: 8337, loss_rpn_cls: 0.0173, loss_rpn_bbox: 0.0339, loss_cls: 0.1831, acc: 94.0222, loss_bbox: 0.2096, loss: 0.4439, grad_norm: 1.7754
2021-05-18 15:19:58,650 - mmdet - INFO - Epoch [19][160/336]	lr: 2.000e-03, eta: 0:30:36, time: 1.030, data_time: 0.017, memory: 8337, loss_rpn_cls: 0.0139, loss_rpn_bbox: 0.0318, loss_cls: 0.1697, acc: 94.3707, l